## Como gerar curvas de aprendizagem

Uma ferramenta de diagnóstico bastante importante para quem trabalha com ciência dos dados, especificamente com aprendizado de máquina, são as curvas de aprendizagem. Elas trazem detalhes de evolução do modelo com relação à exposição de novas amostras e como se comporta o desempenho do conjunto de teste fixo.

---

O conjunto de dados a ser utilizado será idêntico ao da primeira parte desse encontro. Novamente os dados serão carregados e organizados em uma variável de atributo e uma de rótulo, X e y, respectivamente.

In [ ]:
import pandas as pd

df = pd.read_csv('energy_data.csv')
df.describe()

X = df.drop(['Appliances','date'],axis=1)
y = df.Appliances

A curva de aprendizagem pode ser obtida ao executar consecutivamente o ajuste de um modelo a um número progressivo de amostras de treino, medindo seu desempenho contra um conjunto de teste de tamanho fixo. Ao final, é possível utilizar essas informações para verificar como acontece a aprendizagem do modelo.

In [ ]:
from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import seaborn as sns

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.3, random_state=42)

n_amostras = list(range(1, X_train.shape[0], int(X_train.shape[0]/50)))
resultados_treino = []
resultados_teste  = []

# suponha dividir todos os dados em 10 partes
for i in n_amostras:
    model = linear_model.LinearRegression()
    model.fit(X_train[:i], y_train[:i])
    resultados_treino.append(metrics.mean_squared_error(model.predict(X_train[:i]), y_train[:i]))
    resultados_teste.append(metrics.mean_squared_error(model.predict(X_test), y_test))

    
plt.figure()
plt.plot(n_amostras, resultados_treino, 'o-', color="r", label="Treino")
plt.plot(n_amostras, resultados_teste, 'o-', color="b", label="Teste")
plt.legend(loc='best')
plt.show()

Nesse gráfico é possível observar alguns aspectos importantes:
1. No primeiro instante, o treino não tem erro nenhum, e o teste tem um erro altíssimo. Isso é esperado porque o modelo foi treinado com uma ou pouquíssimas amostras, e tem que acertar somente essa pequena quantidade.
2. Rapidamente, o erro cai do teste porque o modelo está generalizando, enquanto o erro do treino sobe porque está aumentando o número de amostras que faz parte da avaliação agora.
3. Em seguida, o erro do treino começa diminuir consideravelmente, mas o erro do teste segue descendo bem pouco.

Esse cenário não é ruim, mas é importante se **questionar se esse erro estacionário próximo a 8k não é muito alto**, e observar **se a tendência da curva não é o erro de teste começar a subir**.

Alternativo a fazer todo o processo manualmente, a função *learning_curve* auxilia a executar o processo de encontrar os diferentes valores de erro para diferentes tamanhos de amostras de treino.

In [ ]:
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn import linear_model

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import seaborn as sns

def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):

    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes,
        scoring='neg_mean_squared_error')
    train_scores_mean = np.mean(train_scores, axis=1) * -1
    train_scores_std = np.std(train_scores, axis=1) * -1
    test_scores_mean = np.mean(test_scores, axis=1) * -1
    test_scores_std = np.std(test_scores, axis=1) * -1
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Treino")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Teste")

    plt.legend(loc="best")
    return plt


title = "Curva de Aprendizagem"
cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)

estimator = linear_model.Lasso(alpha=2.5)
plot_learning_curve(estimator, title, X, y, cv=cv, n_jobs=4)

plt.show()

Os resultados são suavizados com cross-validation no exemplo anterior e é possível perceber uma sobreposição da região verde com a vermelha quase ao final da curva. Quando isso acontece, pode ser interessante parar o treinamento antes ou mudar o modelo para algum outro que não sobreajuste.

## Exercícios

1. **Analise a curva de aprendizagem para as melhores configurações encontradas nos exercícios do notebook anterior.**